# practise the cnn

In [39]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

mnist = input_data.read_data_sets('../data/', one_hot=True)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [40]:
# build the graph: conv1, fc1, softmax
n_input = 28 * 28 * 1
n_classes = 10
x = tf.placeholder(tf.float32, shape=[None, n_input])
y = tf.placeholder(tf.float32, shape=[None, n_classes])

x_ = tf.reshape(x, shape=[-1, 28, 28, 1])

# conv1 layer
conv1_n = 32
filter_size = 5
conv1 = {"weight": tf.Variable(tf.truncated_normal([filter_size, filter_size, 1, conv1_n], stddev=0.1)), 
        "bias": tf.Variable(tf.truncated_normal([1, conv1_n], stddev=0.1))}

conv1_z = tf.nn.conv2d(x_, conv1["weight"], strides=[1, 1, 1, 1], padding="SAME")
conv1_a = tf.nn.relu(conv1_z + conv1["bias"])
# max pool layer
pool1_a = tf.nn.max_pool(conv1_a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
print "conv1 layer with max pool  build done"

# fc layer
fc1_input = tf.reshape(pool1_a, [-1, conv1_n * 14 * 14])
fc1_n = 128
fc1 = {"weight": tf.Variable(tf.truncated_normal([conv1_n * 14 * 14, fc1_n], stddev=0.1)),
      "bias": tf.Variable(tf.truncated_normal([1, fc1_n], stddev=0.1))}

fc1_z = tf.add(tf.matmul(fc1_input, fc1["weight"]), fc1["bias"])
fc1_a = tf.nn.relu(fc1_z)
print "fc layer build done"

# output layer
out = {"weight": tf.Variable(tf.truncated_normal([fc1_n, n_classes], stddev=0.1)),
      "bias": tf.Variable(tf.truncated_normal([1, n_classes], stddev=0.1))}


y_ = tf.nn.softmax(tf.add(tf.matmul(fc1_a, out["weight"]), out["bias"]))

# the cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))

optm = tf.train.AdamOptimizer().minimize(cost)
corr = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, tf.float32))
print "cost and optimzer build done"
print "compute graph ready!"

conv1 layer build done
max pool layer build done
fc layer build done
cost and optimzer build done
compute graph ready!


In [42]:
# run!
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print "init run done"

epochs = 10
batch_size = 128
display_step = 1
total_batch = int(mnist.train.num_examples/batch_size)

writer = tf.summary.FileWriter('/tmp/tf_logs/cnn-mnist', graph=sess.graph)
tf.summary.scalar("accuracy", accr)
summaries = tf.summary.merge_all()

j = 0
for epoch in range(epochs):
    avg_cost = 0.
    print "#Epoch ", epoch
    
    # ITERATION
    for i in range(total_batch):
        j += 1
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        _, percost = sess.run([optm, cost], feed_dict=feeds)
        avg_cost += percost
#         writer.add_summary(summary, j)
    avg_cost = avg_cost / total_batch
    
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc, summary_train = sess.run([accr, summaries], feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
#         writer.add_summary(summary_train, epoch)
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
#         writer.add_summary(test_acc,)
        print ("TEST ACCURACY: %.3f" % (test_acc))
                
print ("OPTIMIZATION FINISHED")

init run done
#Epoch  0
Epoch: 000/010 cost: 1.586331831


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_36' with dtype float
	 [[Node: Placeholder_36 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder_36', defined at:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Library/Python/2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Python/2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Python/2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Library/Python/2.7/site-packages/zmq/eventloop/ioloop.py", li